In [1]:
from torchkf import *
import numpy as np

import plotly.express as px
import plotly.graph_objs as go

import symengine as si

In [2]:
def pendulum_f(x,v,p): 
    q, w = x[:, 0]
    acc  = -p[0, 0] * si.sin(q)
    dx   = si.Matrix([[w], [acc]])
    return dx

models = [
    GaussianModel(
        f=pendulum_f, 
        g=lambda x,v,p: x[0, None], 
        n=2, m=1, pE=np.array([9.81 * 0.1]), p=1, l=1, sv=1/8., sw=1/8.,
        V=np.asarray([np.exp(6)]), W=np.asarray([np.exp(32)] * 2), 
        x=np.asarray([[np.pi/10.], [0]])
    )
]
hdm = HierarchicalGaussianModel(*models, dt=0.1)

Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 0.00s)
  Compiling g... g() ok. (compiled in 0.00s)
Done. 


Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 0.00s)
  Compiling g... g() ok. (compiled in 0.00s)
Done. 


In [19]:
models = [
    GaussianModel(
        f=pendulum_f, 
        g=lambda x,v,p: x[0, None], 
        n=2, m=1, pE=np.array([9.81 * 0.1]), p=1, l=1, sv=1/8., sw=1/8.,
        V=np.asarray([np.exp(6)]), W=np.asarray([np.exp(32)] * 2), 
        x=np.asarray([[np.pi/10.], [0]])
    )
]
hdm = HierarchicalGaussianModel(*models, dt=0.1)
# hdm[0].sw = 1/2.
# hdm[0].sv = 1/2.
# hdm.dt = 0.1
z, _ = dem_z(hdm, 1024)
def autocorr(x):
    x = x.squeeze()
    result = np.correlate(x, x, mode='full')
    return result[int(result.size/2):][:20] / result[result.size//2]
print(z[0].var())
px.line(y=autocorr(z[0]))


Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 0.00s)
  Compiling g... g() ok. (compiled in 0.00s)
Done. 
0.002042217123831625


In [20]:
gen = DEMInversion(hdm).generate(2048)
px.line(y=[gen.x[:, 0, 0, 0], gen.x[:, 0, 1, 0], gen.v[:, 0, 0, 0]])

  0%|          | 0/2048 [00:00<?, ?it/s]

In [28]:
models = [
    GaussianModel(
        f=pendulum_f, 
        g=lambda x,v,p: x[0, None], 
        n=2, m=1, pE=np.array([9.81 * 0.1]), p=1, l=1, sv=1/8, sw=1/8,
        V=np.asarray([np.exp(6)]), W=np.asarray([np.exp(8), np.exp(32)]), 
        x=np.asarray([[np.pi/10.], [0]])
    )
]
hdm = HierarchicalGaussianModel(*models, dt=0.1)

x  = 2 * gen.v[:, 0, :1, 0]
dec = DEMInversion(hdm).run(x)

Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 0.00s)
  Compiling g... g() ok. (compiled in 0.00s)
Done. 


timestep:   0%|          | 0/2048 [00:00<?, ?it/s]

In [29]:
n   = 5
nxe = dec.qU.x.shape[1] * dec.qU.x.shape[2]
nx  = int(nxe / 5)
nve = dec.qU.y.shape[1] * dec.qU.y.shape[2]
nv  = int(nve / 5)

Cxe = dec.qU.c[:, :nxe, :nxe]
Cxe = Cxe.reshape((Cxe.shape[0], n, nx, n, nx))
Cx  = np.einsum('tnini->tni', Cxe)

Cve = dec.qU.c[:, nxe:nxe+nve, nxe:nxe+nve]
Cve = Cve.reshape((Cve.shape[0], n, nv, n, nv))
Cv  = np.einsum('tnini->tni', Cve)

# for i in range(nx):
#     px.imshow(Cxe[128,:,i,:,i]).show()
# for i in range(nv):
#     px.imshow(Cve[0,:,i,:,i]).show()

In [34]:
emb = DEMInversion.generalized_coordinates(gen.v[:, 0, :1, 0], p=2, dt=0.1)
px.line(y=[dec.qU.x[:, 0, 0], dec.qU.x[:, 1, 1], dec.qU.x[:, 2, 0]])#, emb[:, 1, 0]])

In [132]:
np.set_printoptions(linewidth=256, precision=9)

In [133]:
DEMInversion.generalized_covariance(4, 1/8)

array([[ 1.500000000e+00, -0.000000000e+00,  1.562500000e-02, -0.000000000e+00],
       [-0.000000000e+00,  7.812500000e-02, -0.000000000e+00,  4.882812500e-04],
       [ 1.562500000e-02,  0.000000000e+00,  4.882812500e-04,  0.000000000e+00],
       [ 0.000000000e+00,  4.882812500e-04,  0.000000000e+00,  5.086263021e-06]])